# Cleaning data Kennisinstituut

<span style="color:red">**Still Missing: Retrieving pubmed ID or DOI. And screening conclusions that are not in the notes column but jumped around need to be traced in the files before throwing away columns.**</span>

## Data extraction from csv files

In this part we will extract data from the csv files that are exported from rayyan. 

First load in the packages

In [21]:
# import packages
import os
import pandas as pd
import numpy as np # for np.nan
import re # for searching in notes column

Get all csv files from the folder

In [2]:
# List all .csv files in the current directory
csv_files = [file for file in os.listdir('.') if file.endswith('.csv')]

# Display the list of .csv files
print(csv_files)

['articles_cataract_IOL.csv', 'articles_chronischebekkenpijn_botoxinjectie.csv', 'articles_coeliakie_NCGS.csv', 'articles_COPD_tripletherapy.csv', 'articles_DDH_Pavlik.csv', 'articles_dehydratiekinderen_antiemetica.csv', 'articles_distaleradiusfracturen_operatiefvsconservatief.csv', 'articles_downsyndroom_vruchtbaarheid.csv', 'articles_geboortezorg_agraves.csv', 'articles_HB_HI_fluoxetine.csv', 'articles_HHT_endoscopische_chirurgie .csv', 'articles_influenza_treatment.csv', 'articles_PA_gonadenverwijderen.csv', 'articles_SAB_vochtbeleid.csv', 'articles_schisis_timing_techniek.csv', 'articles_schouderprothese_totaalvshemi.csv', 'articles_sectiofoetalenood.csv', 'articles_septumcorrectie_kinderen.csv', 'articles_uveitis_biologicals.csv', 'articles_vergroot_ovarium_chirurgie.csv']


Figure out how many colums each file has (are they equal? -> No). Then find the common column names so we can extract the relevant data from the files.

In [8]:
ncols = [] # number of columns in each file
column_sets = []  # List to store the set of columns from each file

for file_name in csv_files:
    df = pd.read_csv(file_name, sep=';', engine='python')
    ncols.append(df.shape[1]) # Append the number of columns to the list
    
    # Append the set of column names to the list
    column_sets.append(set(df.columns))

print(ncols)

# Find columns present in all files
common_columns = set.intersection(*column_sets)
print("Columns present in all files:", common_columns)

[113, 20, 19, 25, 19, 34, 35, 19, 36, 19, 19, 21, 19, 206, 49, 21, 25, 20, 19, 99]
Columns present in all files: {'title', 'issue', 'key', 'language', 'authors', 'month', 'volume', 'abstract', 'pubmed_id', 'url', 'publisher', 'journal', 'year', 'notes', 'issn', 'pages', 'day', 'pmc_id', 'location'}


### Removing irrelevant columns
Not all the common columns are relevant so after looking into the content we remove some columns that we don't need to select.

Columns that are removes are:
- authors
- issue
- key
- language
- month
- volume
- publisher
- journal
- issn
- pages
- day
- pmc_id
- location
- year


In [14]:
# Read the CSV file with semicolon separator
df = pd.read_csv(csv_files[1], sep=';', engine='python')
# extract relevan columns
columns = ['title', 'abstract', 'pubmed_id', 'url', 'notes']
# select the relevant columns
df_selected = df[columns]
# display the first 5 rows
print(df_selected.head())

                                               title  \
0  Cystoscopic evaluation and clinical phenotypin...   
1     Interstitial cystitis - intravesical treatment   
2  Pharmacotherapy of interstitial cystitis in women   
3  Recommendations on the Use of Botulinum Toxin ...   
4  Botulinum toxin treatment of pelvic floor diso...   

                                            abstract  pubmed_id  \
0  Herein, we aimed to review, report, and discus...        NaN   
1  The Guidelines Project, an initiative of the B...        NaN   
2  Interstitial cystitis is a condition that affe...        NaN   
3  Context: The increasing body of evidence and n...        NaN   
4  Background and Objective: Botulinum neurotoxin...        NaN   

                                                 url  \
0  http://www.embase.com/search/results?subaction...   
1  http://ovidsp.ovid.com/ovidweb.cgi?T=JS&PAGE=r...   
2  http://www.embase.com/search/results?subaction...   
3  http://www.embase.com/search/resu

## What we want to get
What we want to extract in the end is 
- title	
- abstract	
- doi	
- pmid	
- TI-AB (title-abstract inclusion status)
- FT (full-text inclusion status)
- GI (guideline inclusion status)

Where we can we want to extract as much as possible. Title and abstract will be one to one transferable. DOI or pmid will have to be retrieved somehow, this will require a process of retrieval. From the inclusion status only TI-AB is known now and will have to be taken from the notes column of the csv files. That will require cleaning step within the data.


In [17]:
# Function to extract content within {} after "RAYYAN-INCLUSION:"
def extract_inclusion_status(note):
    match = re.search(r'RAYYAN-INCLUSION:\s*({.*?})', note)
    return match.group(1) if match else None

# Apply the function to the 'notes' column
df_selected['inclusion_status'] = df_selected['notes'].apply(lambda x: extract_inclusion_status(x) if pd.notnull(x) else None)

# Display the first 5 rows with the new column
print(df_selected[['notes', 'inclusion_status']].head())

                                               notes  \
0                                                NaN   
1   RAYYAN-INCLUSION: {"Dick"=>"Maybe", "k.j.schw...   
2  L46157659     2007-02-27 RAYYAN-INCLUSION: {"D...   
3  L50273447     2008-12-17 RAYYAN-INCLUSION: {"D...   
4  L352487028     2008-10-21 RAYYAN-INCLUSION: {"...   

                                    inclusion_status  
0                                               None  
1       {"Dick"=>"Maybe", "k.j.schweitzer"=>"Maybe"}  
2  {"Dick"=>"Excluded", "k.j.schweitzer"=>"Exclud...  
3  {"Dick"=>"Included", "k.j.schweitzer"=>"Includ...  
4  {"Dick"=>"Included", "k.j.schweitzer"=>"Includ...  


C:\Users\5507553\AppData\Local\Temp\ipykernel_12816\1624620693.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['inclusion_status'] = df_selected['notes'].apply(lambda x: extract_inclusion_status(x) if pd.notnull(x) else None)


In [18]:
# Function to map decisions to codes
def map_decision(decision):
    if decision.lower() == "excluded":
        return 0
    elif decision.lower() == "included":
        return 1
    elif decision.lower() == "maybe":
        return 999
    else:
        return None

# Function to extract names and coded decisions from inclusion_status
def extract_decisions(inclusion_status):
    if pd.isnull(inclusion_status):
        return None
    decisions = re.findall(r'"(.*?)"\s*=>\s*"(.*?)"', inclusion_status)
    return {name: map_decision(decision) for name, decision in decisions}

# Apply the function to the 'inclusion_status' column
df_selected['coded_decisions'] = df_selected['inclusion_status'].apply(extract_decisions)

# Display the first 5 rows with the new column
print(df_selected[['inclusion_status', 'coded_decisions']].head())

                                    inclusion_status  \
0                                               None   
1       {"Dick"=>"Maybe", "k.j.schweitzer"=>"Maybe"}   
2  {"Dick"=>"Excluded", "k.j.schweitzer"=>"Exclud...   
3  {"Dick"=>"Included", "k.j.schweitzer"=>"Includ...   
4  {"Dick"=>"Included", "k.j.schweitzer"=>"Includ...   

                        coded_decisions  
0                                  None  
1  {'Dick': 999, 'k.j.schweitzer': 999}  
2      {'Dick': 0, 'k.j.schweitzer': 0}  
3      {'Dick': 1, 'k.j.schweitzer': 1}  
4      {'Dick': 1, 'k.j.schweitzer': 1}  


C:\Users\5507553\AppData\Local\Temp\ipykernel_12816\2527196334.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['coded_decisions'] = df_selected['inclusion_status'].apply(extract_decisions)


### Decision on TI-AB classification
After discussion between Tim and Duco we decided that the best "representation" of the screening process is:
- Two exclusions will be exclusion
- anything else represents doubt and thus the article will go through to a next phase in screening, thus inclusion.


In [22]:
# Create the TI-AB column based on the coded_decisions
df_selected['TI-AB'] = df_selected['coded_decisions'].apply(
    lambda decisions: np.nan if decisions is None or decisions == 'None' else (
        0 if decisions and all(decision == 0 for decision in decisions.values()) else 1
    )
)

# Display the first 5 rows with the new column
print(df_selected[['coded_decisions', 'TI-AB']].head())


                        coded_decisions  TI-AB
0                                  None    NaN
1  {'Dick': 999, 'k.j.schweitzer': 999}    1.0
2      {'Dick': 0, 'k.j.schweitzer': 0}    0.0
3      {'Dick': 1, 'k.j.schweitzer': 1}    1.0
4      {'Dick': 1, 'k.j.schweitzer': 1}    1.0


C:\Users\5507553\AppData\Local\Temp\ipykernel_12816\2756579335.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['TI-AB'] = df_selected['coded_decisions'].apply(


## clean export

Now we need to cleanup the 'df_selected' DataFrame and export the relevant information.

In [23]:
# Select the relevant columns from df_selected
df_final = df_selected[['title', 'abstract', 'pubmed_id', 'url', 'TI-AB']]

# Display the first 5 rows of the final dataframe
print(df_final.head())

                                               title  \
0  Cystoscopic evaluation and clinical phenotypin...   
1     Interstitial cystitis - intravesical treatment   
2  Pharmacotherapy of interstitial cystitis in women   
3  Recommendations on the Use of Botulinum Toxin ...   
4  Botulinum toxin treatment of pelvic floor diso...   

                                            abstract  pubmed_id  \
0  Herein, we aimed to review, report, and discus...        NaN   
1  The Guidelines Project, an initiative of the B...        NaN   
2  Interstitial cystitis is a condition that affe...        NaN   
3  Context: The increasing body of evidence and n...        NaN   
4  Background and Objective: Botulinum neurotoxin...        NaN   

                                                 url  TI-AB  
0  http://www.embase.com/search/results?subaction...    NaN  
1  http://ovidsp.ovid.com/ovidweb.cgi?T=JS&PAGE=r...    1.0  
2  http://www.embase.com/search/results?subaction...    0.0  
3  http://ww

In [ ]:
# Export the df_final DataFrame to a CSV file with the modified name
output_file_name = file_name.replace('.csv', '_CLEAN_TRAM.csv')
df_final.to_csv(output_file_name, index=False)

# Display the name of the output file
print(f"DataFrame exported to {output_file_name}")